## **CS 6120: Natural Language Processing - Prof. Ahmad Uzair** 

### **Assignment 3: Parts Of Speech Tagging**

### **Total points: 100**

In this assignment, you will have hands-on experience with HMMs and Viterbi algorithm. You'll will be doing part-of-speech (POS) tagging, the process of assigning a part-of-speech tag (Noun, Verb, Adjective...) to each word in an input text.

In [ ]:
import pandas as pd
from collections import defaultdict
import math
import numpy as np
import string

We have got the preprocessing covered for you in this assignment

In [ ]:
# Do not change anything in this cell

# Punctuation characters
punct = set(string.punctuation)

# Morphology rules used to assign unknown word tokens
noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
verb_suffix = ["ate", "ify", "ise", "ize"]
adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
adv_suffix = ["ward", "wards", "wise"]


def get_word_tag(line, vocab): 
    if not line.split():
        word = "--n--"
        tag = "--s--"
        return word, tag
    else:
        word, tag = line.split()
        if word not in vocab: 
            # Handle unknown words
            word = assign_unk(word)
        return word, tag
    return None 


def preprocess(vocab, data_fp):
    """
    Preprocess data
    """
    orig = []
    prep = []

    # Read data
    with open(data_fp, "r") as data_file:

        for cnt, word in enumerate(data_file):

            # End of sentence
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            # Handle unknown words
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word)
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep


def assign_unk(tok):
    """
    Assign unknown word tokens
    """
    # Digits
    if any(char.isdigit() for char in tok):
        return "--unk_digit--"

    # Punctuation
    elif any(char in punct for char in tok):
        return "--unk_punct--"

    # Upper-case
    elif any(char.isupper() for char in tok):
        return "--unk_upper--"

    # Nouns
    elif any(tok.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"

    # Verbs
    elif any(tok.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"

    # Adjectives
    elif any(tok.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"

    # Adverbs
    elif any(tok.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"

    return "--unk--"

## Introduction to Dataset
 
- This assignment will use two tagged data sets. One data set (**train.pos**) will be used for **training**. The other (**test.pos**) for **testing**. The tagged training data has been preprocessed to form a vocabulary (**hmv.txt**). The words in the vocabulary are words from the training set that were used two or more times. The vocabulary is augmented with a set of 'unknown word tokens'. 

- The training set will be used to create the emission, transmission and tag counts. 

- The test set (test.pos) is read in to create `y`. This contains both the test text and the true tag. The test set has also been preprocessed to remove the tags to form **test_words.txt**. 

- A POS tagger will necessarily encounter words that are not in its datasets.  A set of unknown-tokens, such as '--unk-verb--' or '--unk-noun--' will replace the unknown words in both the training and test corpus and will appear in the emission, transmission and tag data structures.

### **Task 1 - Training the model: (20 points)**

Write a program that takes in the `training_corpus` and returns the three dictionaries mentioned above `transition_counts`, `emission_counts`, and `tag_counts`. 
- `emission_counts`: maps (tag, word) to the number of times it happened. 
- `transition_counts`: maps (prev_tag, tag) to the number of times it has appeared. 
- `tag_counts`: maps (tag) to the number of times it has occured.

Tip: Make use of defaultdict as a standard Python dictionary throws a *KeyError* if you try to access an item with a key that is not currently in the dictionary. In contrast, the *defauweltdict* will create an item of the type of the argument, in this case an integer with the default value of 0.

In [ ]:
# Do not change anything in this cell
# load in the training corpus
with open("train.pos", 'r') as f:
    training_corpus = f.readlines()

# read the vocabulary data, split by each line of text, and save the list
with open("hmv.txt", 'r') as f:
    voc_l = f.read().split('\n')

# vocab: dictionary that has the index of the corresponding words
vocab = {} 

# Get the index of the corresponding words. 
for i, word in enumerate(sorted(voc_l)): 
    vocab[word] = i       
    

cnt = 0
for k,v in vocab.items():
    cnt += 1
    if cnt > 20:
        break

# load in the test corpus
with open("test.pos", 'r') as f:
    y = f.readlines()

#corpus without tags, preprocessed
_, prep = preprocess(vocab, "test.words")

In [ ]:
# TASK 1 CELL

def create_dictionaries(training_corpus, vocab):
    """
    Params: 
        training_corpus: a corpus where each line has a word followed by its tag.
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Return: 
        emission_counts: a dictionary where the keys are (tag, word) and the values are the counts
        transition_counts: a dictionary where the keys are (prev_tag, tag) and the values are the counts
        tag_counts: a dictionary where the keys are the tags and the values are the counts
    """
    
    # initialize the dictionaries 
    emission_counts=dict()
    
    for i in training_corpus:
        word,tag=get_word_tag(i, vocab)
        #if tag=='``':
            #continue
        if (tag,word) not in emission_counts:
            emission_counts[(tag,word)]=1
        else:
            emission_counts[(tag,word)]+=1
            
            
    transition_counts=dict()
    
    for i in range(len(training_corpus)-1):
        this_tag=get_word_tag(training_corpus[i], vocab)[1]
        next_tag=get_word_tag(training_corpus[i+1], vocab)[1]
        #if this_tag=='``' or next_tag=='``':
            #continue
        if (this_tag,next_tag) not in transition_counts:
            transition_counts[(this_tag,next_tag)]=1
        else:
            transition_counts[(this_tag,next_tag)]+=1
            
        
    tag_counts=dict()
    
    for i in training_corpus:
        word,tag=get_word_tag(i, vocab)
        if tag not in tag_counts:
            tag_counts[tag]=1
        else:
            tag_counts[tag]+=1
        
        
    # Initialize "prev_tag" (previous tag) with the start state, denoted by '--s--'

        # get the word and tag using the get_word_tag helper function
        
    return emission_counts, transition_counts, tag_counts

In [ ]:
# Do not change anything in this cell

emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)
states = sorted(tag_counts.keys())
print(f"Number of POS tags (number of 'states'): {len(states)}")
print("View these POS tags (states)")
print(states)

### **Task 2: Testing the model: (10 points)**
Implement `predict_pos` that computes the accuracy of your model.
- To assign a part of speech to a word, assign the most frequent POS for that word in the training set. 
- Then evaluate how well this approach works.  Each time you predict based on the most frequent POS for the given word, check whether the actual POS of that word is the same.  If so, the prediction was correct!
- Calculate the accuracy as the number of correct predictions divided by the total number of words for which you predicted the POS tag. 

In [ ]:
# Task 2 Cell
def predict_pos(prep, y, emission_counts, vocab, states):
    '''
    Params: 
        prep: a preprocessed version of 'y'. A list with the 'word' component of the tuples.
        y: a corpus composed of a list of tuples where each tuple consists of (word, POS)
        emission_counts: a dictionary where the keys are (tag,word) tuples and the value is the count
        vocab: a dictionary where keys are words in vocabulary and value is an index
        states: a sorted list of all possible tags for this assignment
    Return: 
        accuracy: Number of times you classified a word correctly
    '''

    num=len(y)
  
    y_tup=[]
    for line in y:
        if not line.split():
            continue
        else:
            word, pos = line.split()
            if word not in vocab: 
                continue
        word,pos=get_word_tag(line,vocab)
        y_tup.append([word,pos])

    L=[]
    for i in range(num):
        max_count=0
        max_tag=None
        for key in list(emission_counts.keys()):
            if key[1]==prep[i]:
                if emission_counts[key] > max_count:
                    max_count=emission_counts[key]
                    max_tag=key[0]
        if [prep[i],max_tag] not in L:
            L.append([prep[i],max_tag])
    
    L=sorted(L,key=lambda x: x[0])
    #print(L)
    y_tup=sorted(y_tup,key=lambda x: x[0])
    correct=0
    total=0
    for i in range(len(L)):
        if L[i][1] not in vocab:
            continue
        for j in range(len(y_tup)):
            if L[i][0]==y_tup[j][0]:
                if y_tup[j][1]==L[i][1]:
                    correct+=1
                total+=1
    accuracy=correct/total
    return accuracy

In [ ]:
# Do not change anything in this cell
accuracy_predict_pos = predict_pos(prep, y, emission_counts, vocab, states)
print(f"Accuracy of prediction using predict_pos is {accuracy_predict_pos:.4f}")

### **Task 3 - Hidden Markov Model: (20 points)**

Hidden Markov Model: The HMM is one of the most commonly used algorithms in Natural Language Processing. The Markov Model contains a number of states and the probability of transition between those states. In this case, the states are the parts-of-speech. A Markov Model utilizes a transition matrix, `A`. A Hidden Markov Model adds an observation or emission matrix `B` which describes the probability of a visible observation when we are in a particular state. In this case, the emissions are the words in the corpus. The state, which is hidden, is the POS tag of that word.

The smoothing was done as follows: 

$$ P(t_i | t_{i-1}) = \frac{C(t_{i-1}, t_{i}) + \alpha }{C(t_{i-1}) +\alpha * N}\tag{3}$$

- $N$ is the total number of tags
- $C(t_{i-1}, t_{i})$ is the count of the tuple (previous POS, current POS) in `transition_counts` dictionary.
- $C(t_{i-1})$ is the count of the previous POS in the `tag_counts` dictionary.
- $\alpha$ is a smoothing parameter.

Implement the `create_transition_matrix` below for all tags. 

In [ ]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
    ''' 
    Params: 
        alpha: number used for smoothing
        tag_counts: a dictionary mapping each tag to its respective count
        transition_counts: transition count for the previous word and tag
    Return:
        A: matrix of dimension (num_tags,num_tags)
    '''
    # Write your code here
    
    states = sorted(tag_counts.keys())
    
    matrix=[[0 for i in range(len(states))] for j in range(len(states))]
    for i in range(len(states)):
        for j in range(len(states)):
            pre_tag=states[i]
            next_tag=states[j]
            if (pre_tag,next_tag) in transition_counts:
                matrix[i][j]=(transition_counts[(pre_tag,next_tag)]+alpha)/(tag_counts[pre_tag]+alpha*len(states))
            else:
                matrix[i][j]=1/len(states)
    return np.matrix(matrix)

In [ ]:
# Do not change anything in this cell
alpha = 0.001
A = create_transition_matrix(alpha, tag_counts, transition_counts)
# Testing your function
print(f"A at row 0, col 0: {A[0,0]:.9f}")
print(f"A at row 3, col 1: {A[3,1]:.4f}")

print("View a subset of transition matrix A")
A_sub = pd.DataFrame(A[30:35,30:35], index=states[30:35], columns = states[30:35] )
print(A_sub)

Now you will create the `B` transition matrix which computes the emission probability. 

You will use smoothing as defined below: 

$$P(w_i | t_i) = \frac{C(t_i, word_i)+ \alpha}{C(t_{i}) +\alpha * N}\tag{4}$$

- $C(t_i, word_i)$ is the number of times $word_i$ was associated with $tag_i$ in the training data (stored in `emission_counts` dictionary).
- $C(t_i)$ is the number of times $tag_i$ was in the training data (stored in `tag_counts` dictionary).
- $N$ is the number of words in the vocabulary
- $\alpha$ is a smoothing parameter. 

The matrix `B` is of dimension (num_tags, N), where num_tags is the number of possible parts-of-speech tags. 

Implement the `create_emission_matrix` below that computes the `B` emission probabilities matrix. Your function takes in $\alpha$, the smoothing parameter, `tag_counts`, which is a dictionary mapping each tag to its respective count, the `emission_counts` dictionary where the keys are (tag, word) and the values are the counts.

In [ ]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    '''
    Params: 
        alpha: tuning parameter used in smoothing 
        tag_counts: a dictionary mapping each tag to its respective count
        emission_counts: a dictionary where the keys are (tag, word) and the values are the counts
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Return:
        B: a matrix of dimension (num_tags, len(vocab))
    '''
    
    # Write your code here
    #tag_counts=[i for i in tag_counts.items()]
    states = sorted(tag_counts.keys())
    
    vocab_list=list(vocab.items())

    num_tags=len(tag_counts)
    matrix=[[0 for i in range(len(vocab)+8)] for j in range(num_tags)]
    for i in range(num_tags):
        #tag=tag_counts[i][0]
        tag=states[i]
        #c_tag=tag_counts[i][1] 
        c_tag=tag_counts[states[i]]
        
        for j in range(len(vocab_list)):
            if (tag,vocab_list[j][0]) in emission_counts: 
                c_tag_word=emission_counts[(tag,vocab_list[j][0])]
                matrix[i][j]=(c_tag_word+alpha)/(c_tag+alpha*len(vocab))
            else:
                matrix[i][j]=1/len(vocab)
        if (tag,"--unk_digit--") in emission_counts:
            c_tag_word=emission_counts[(tag,"--unk_digit--")]
            matrix[i][len(vocab_list)]=(c_tag_word+alpha)/(c_tag+alpha*len(vocab))
        else:
            matrix[i][len(vocab_list)]=1/len(vocab)
        if (tag,"--unk_punct--") in emission_counts:
            c_tag_word=emission_counts[(tag,"--unk_punct--")]
            matrix[i][len(vocab_list)+1]=(c_tag_word+alpha)/(c_tag+alpha*len(vocab))
        else:
            matrix[i][len(vocab_list)+1]=1/len(vocab)
        if (tag,"--unk_upper--") in emission_counts:
            c_tag_word=emission_counts[(tag,"--unk_upper--")]
            matrix[i][len(vocab_list)+2]=(c_tag_word+alpha)/(c_tag+alpha*len(vocab))
        else:
            matrix[i][len(vocab_list)+2]=1/len(vocab)
        if (tag,"--unk_noun--") in emission_counts:
            c_tag_word=emission_counts[(tag,"--unk_noun--")]
            matrix[i][len(vocab_list)+3]=(c_tag_word+alpha)/(c_tag+alpha*len(vocab))
        else:
            matrix[i][len(vocab_list)+3]=1/len(vocab)
        if (tag,"--unk_verb--") in emission_counts:
            c_tag_word=emission_counts[(tag,"--unk_verb--")]
            matrix[i][len(vocab_list)+4]=(c_tag_word+alpha)/(c_tag+alpha*len(vocab))
        else:
            matrix[i][len(vocab_list)+4]=1/len(vocab)
        if (tag,"--unk_adj--") in emission_counts:
            c_tag_word=emission_counts[(tag,"--unk_adj--")]
            matrix[i][len(vocab_list)+5]=(c_tag_word+alpha)/(c_tag+alpha*len(vocab))
        else:
            matrix[i][len(vocab_list)+5]=1/len(vocab)
        if (tag,"--unk_adv--") in emission_counts:
            c_tag_word=emission_counts[(tag,"--unk_adv--")]
            matrix[i][len(vocab_list)+6]=(c_tag_word+alpha)/(c_tag+alpha*len(vocab))
        else: 
            matrix[i][len(vocab_list)+6]=1/len(vocab)
        if (tag,"--unk--") in emission_counts:
            c_tag_word=emission_counts[(tag,"--unk--")]
            matrix[i][len(vocab_list)+7]=(c_tag_word+alpha)/(c_tag+alpha*len(vocab))
        else:
            matrix[i][len(vocab_list)+7]=1/len(vocab)

    return np.matrix(matrix)

In [ ]:
# Do not change anything in this cell
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab))
print(f"View Matrix position at row 0, column 0: {B[0,0]:.9f}")
print(f"View Matrix position at row 3, column 1: {B[3,1]:.9f}")
cidx  = ['725','adroitly','engineers', 'promoted', 'synergy']
cols = [vocab[a] for a in cidx]
rvals =['CD','NN','NNS', 'VB','RB','RP']
rows = [states.index(a) for a in rvals]
B_sub = pd.DataFrame(B[np.ix_(rows,cols)], index=rvals, columns = cidx )
print(B_sub)

### **Task 4 - Viterbi Algorithm: (50 points)**

In this part of the assignment you will implement the Viterbi algorithm which makes use of dynamic programming. Specifically, you will use your two matrices, `A` and `B` to compute the Viterbi algorithm. We have decomposed this process into three main steps for you. 

* **Initialization** - In this part you initialize the `best_paths` and `best_probabilities` matrices that you will be populating in `feed_forward`.
* **Feed forward** - At each step, you calculate the probability of each path happening and the best paths up to that point. 
* **Feed backward**: This allows you to find the best path with the highest probabilities. 

You will start by initializing two matrices of the same dimension. 

- best_probs: Each cell contains the probability of going from one POS tag to a word in the corpus.

- best_paths: A matrix that helps you trace through the best possible path in the corpus. 

 
Write a program below that initializes the `best_probs` and the `best_paths` matrix. 

Both matrices will be initialized to zero except for column zero of `best_probs`.  
- Column zero of `best_probs` is initialized with the assumption that the first word of the corpus was preceded by a start token ("--s--"). 
- This allows you to reference the **A** matrix for the transition probability

Here is how to initialize column 0 of `best_probs`:
- The probability of the best path going from the start index to a given POS tag indexed by integer $i$ is denoted by $\textrm{best_probs}[s_{idx}, i]$.
- This is estimated as the probability that the start tag transitions to the POS denoted by index $i$: $\mathbf{A}[s_{idx}, i]$ AND that the POS tag denoted by $i$ emits the first word of the given corpus, which is $\mathbf{B}[i, vocab[corpus[0]]]$.

Conceptually, it looks like this:
$\textrm{best_probs}[s_{idx}, i] = \mathbf{A}[s_{idx}, i] \times \mathbf{B}[i, corpus[0] ]$


In order to avoid multiplying and storing small values on the computer, we'll take the log of the product, which becomes the sum of two logs:

$best\_probs[i,0] = log(A[s_{idx}, i]) + log(B[i, vocab[corpus[0]]$

Also, to avoid taking the log of 0 (which is defined as negative infinity), the code itself will just set $best\_probs[i,0] = float('-inf')$ when $A[s_{idx}, i] == 0$


So the implementation to initialize $best\_probs$ looks like this:

$ if A[s_{idx}, i] <> 0 : best\_probs[i,0] = log(A[s_{idx}, i]) + log(B[i, vocab[corpus[0]]])$

$ if A[s_{idx}, i] == 0 : best\_probs[i,0] = float('-inf')$



In [ ]:
# UNQ_C5 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: initialize
def initialize(states, tag_counts, A, B, corpus, vocab):
    '''
    Params: 
        states: a list of all possible parts-of-speech
        tag_counts: a dictionary mapping each tag to its respective count
        A: Transition Matrix of dimension (num_tags, num_tags)
        B: Emission Matrix of dimension (num_tags, len(vocab))
        corpus: a sequence of words whose POS is to be identified in a list 
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Return:
        best_probs: matrix of dimension (num_tags, len(corpus)) of floats
        best_paths: matrix of dimension (num_tags, len(corpus)) of integers
    '''
    # Write your code here 
    num_tags=len(tag_counts)
    best_probs=np.matrix([[0.00 for i in range(len(corpus))] for j in range(num_tags)])
    best_paths=np.matrix([[0.00 for i in range(len(corpus))] for j in range(num_tags)])
    return best_probs, best_paths

In [ ]:
# Do not change anything in this cell

best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)
print(f"best_probs[0,0]: {best_probs[0,0]:.4f}") 
print(f"best_paths[2,3]: {best_paths[2,3]:.4f}")

Implement the `viterbi_forward` algorithm and store the best_path and best_prob for every possible tag for each word in the matrices `best_probs` and `best_tags` using the pseudo code below.

`for each word in the corpus

    for each POS tag type that this word may be
    
        for POS tag type that the previous word could be
        
            compute the probability that the previous word had a given POS tag, that the current word has a given POS tag, and that the POS tag would emit this current word.
            
            retain the highest probability computed for the current word
            
            set best_probs to this highest probability
            
            set best_paths to the index 'k', representing the POS tag of the previous word which produced the highest probability `

In [ ]:
# TASK CELL
def viterbi_forward(A, B, test_corpus, best_probs, best_paths, vocab):
    '''
    Input: 
        A, B: The transiton and emission matrices respectively
        test_corpus: a list containing a preprocessed corpus
        best_probs: an initilized matrix of dimension (num_tags, len(corpus))
        best_paths: an initilized matrix of dimension (num_tags, len(corpus))
        vocab: a dictionary where keys are words in vocabulary and value is an index 
    Output: 
        best_probs: a completed matrix of dimension (num_tags, len(corpus))
        best_paths: a completed matrix of dimension (num_tags, len(corpus))
    '''

    num_tags = best_probs.shape[0]
    
    vocab_list=list(vocab.items())
    
    for i in range(len(states)):
        if states[i]=="--s--":
            pos=i
    p=vocab[test_corpus[0]]
    #print('p: ',p)
    for x in range(num_tags):
        if A[pos,x]==0 or B[x,p]==0:
            best_probs[x,0]=float('-inf')
        else:
            best_probs[x,0]=np.log(A[pos,x])+np.log(B[x,p]) 
        #best_probs[x,0]=1.000
        #print('A[pos,x]: ',A[pos,x],'B[x,p]: ',B[x,p])
    #for x in range(num_tags):
        #print(best_probs[x,0])
    
    for i in range(1, len(test_corpus)): 
        if i % 5000 == 0:
            print("Words processed: {:>8}".format(i))
        for c in range(num_tags):
            max_value=float('-inf')
            max_pre_tag=-1
            pos=vocab[test_corpus[i]]
            if test_corpus[i] not in vocab:
                pos=assign_unk(test_corpus[i]) 
            for a in range(num_tags):
                #print('A[a,c]: ',A[a,c])
                if best_probs[a,i-1]+np.log(A[a,c])>max_value:
                    max_value=best_probs[a,i-1]+np.log(A[a,c])
                    #print('max_value: ',max_value)
                    max_pre_tag=a
                    #print('B[c,pos]: ',B[c,pos])
                    #print('max_value*B[c,pos]: ',float(max_value*B[c,pos]))
            best_probs[c,i]=max_value+np.log(B[c,pos])
            #print('best_probs[c,i]: ',best_probs[c,i])
            best_paths[c,i]=max_pre_tag
            #print(float(max_value*B[c,pos]))
        
    return best_probs, best_paths

In [ ]:
# Do not change anything in this cell
# this will take a few minutes to run => processes ~ 30,000 words
best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, vocab)

In [ ]:
# Do not change anything in this cell
print(f"best_probs[0,1]: {best_probs[0,1]:.4f}") 
print(f"best_probs[0,4]: {best_probs[0,4]:.4f}")

Implement the `viterbi_backward` algorithm, which returns a list of predicted POS tags for each word in the corpus.

- Note that the numbering of the index positions starts at 0 and not 1. 
- `m` is the number of words in the corpus.  
    - So the indexing into the corpus goes from `0` to `m - 1`.
    - Also, the columns in `best_probs` and `best_paths` are indexed from `0` to `m - 1`


**In Step 1:**       
Loop through all the rows (POS tags) in the last entry of `best_probs` and find the row (POS tag) with the maximum value.
Convert the unique integer ID to a tag (a string representation) using the dictionary `states`.  

Referring to the three-word corpus described above:
- `z[2] = 28`: For the word 'upward' at position 2 in the corpus, the POS tag ID is 28.  Store 28 in `z` at position 2.
- states(28) is 'RB': The POS tag ID 28 refers to the POS tag 'RB'.
- `pred[2] = 'RB'`: In array `pred`, store the POS tag for the word 'upward'.

**In Step 2:**  
- Starting at the last column of best_paths, use `best_probs` to find the most likely POS tag for the last word in the corpus.
- Then use `best_paths` to find the most likely POS tag for the previous word. 
- Update the POS tag for each word in `z` and in `preds`.

Referring to the three-word example from above, read best_paths at column 2 and fill in z at position 1.  
`z[1] = best_paths[z[2],2]`  

The small test following the routine prints the last few words of the corpus and their states to aid in debug.

In [ ]:
# TASK CELL
def viterbi_backward(best_probs, best_paths, corpus, states):
    '''
    This function returns the best path.
    
    '''
    # Write your code here
    final_tag_index=list(best_probs[:,-1]).index(max(best_probs[:,-1]))
    trace=[final_tag_index]
    t=final_tag_index
    for i in range(len(prep)-1,0,-1): 
        tag_index=int(best_paths[t,i])
        trace.append(tag_index)
        t=tag_index
    trace=trace[::-1]
    
    #tag_counts=[i for i in tag_counts.items()]
    
    pred=[]
    for index in trace:
        tag=states[index]
        pred.append(tag)
    return pred

In [ ]:
# Do not change anything in cell

pred = viterbi_backward(best_probs, best_paths, prep, states)
m=len(pred)
print('The prediction for pred[-7:m-1] is: \n', prep[-7:m-1], "\n", pred[-7:m-1], "\n")
print('The prediction for pred[0:8] is: \n', pred[0:7], "\n", prep[0:7])

print('The third word is:', prep[3])
print('Your prediction is:', pred[3])
print('Your corresponding label y is: ', y[3])

Implement a function to compute the accuracy of the viterbi algorithm's POS tag predictions.

In [ ]:
# TASK CELL

def compute_accuracy(pred, y):
    '''
    Params: 
        pred: a list of the predicted parts-of-speech 
        y: a list of lines where each word is separated by a '\t' (i.e. word \t tag)
    Return: 
        accuracy
        
    '''
    correct=0
    total=0
    for i in range(len(y)):
        if not y[i].split():
            word = "--n--"
            tag = "--s--"
        else:
            word, tag = y[i].split()
            total+=1
        if pred[i]==tag:
            correct+=1
        
    accuracy=correct/total
        
    return accuracy

In [ ]:
# Do not change anything in cell
print(f"Accuracy of the Viterbi algorithm is {compute_accuracy(pred, y):.4f}")

### References

- ["Speech and Language Processing", Dan Jurafsky and James H. Martin](https://web.stanford.edu/~jurafsky/slp3/)
